In [ ]:
# First reset the pygkyl library
import sys
!rm -rf ~/personal_gkyl_scripts/pygkyl/pygkyl.egg-info
!rm -rf ~/personal_gkyl_scripts/pygkyl/build
!{sys.executable} -m pip install ~/personal_gkyl_scripts/pygkyl > ~/personal_gkyl_scripts/pygkyl/install.log

import numpy as np
import matplotlib.pyplot as plt
import os

# Configure plotting
plt.rcParams["figure.figsize"] = (5,4)

# Custom libraries and routines
import pygkyl

home_dir = os.path.expanduser("~")
repo_dir = home_dir+'/personal_gkyl_scripts/'
simdir = '/Users/ahoffman/personal_gkyl_scripts/sim_data/3x2v/gk_tcv_adapt_src/'
fileprefix = 'rt_gk_tcv_nt_iwl_3x2v_p1'
simulation = pygkyl.simulation_configs.import_config('tcv_nt', simdir, fileprefix)
#simulation.normalization.set('t','mus') # time in micro-seconds
#simulation.normalization.set('x','minor radius') # radial coordinate normalized by the minor radius (rho=r/a)
#simulation.normalization.set('y','Larmor radius') # binormal in term of reference sound Larmor radius
#simulation.normalization.set('z','pi') # parallel angle devided by pi
simulation.normalization.set('fluid velocities','thermal velocity') # fluid velocity moments are normalized by the thermal velocity
simulation.normalization.set('temperatures','eV') # temperatures in electron Volt
simulation.normalization.set('pressures','Pa') # pressures in Pascal
simulation.normalization.set('energies','MJ') # energies in mega Joules
simulation.normalization.set('gradients','major radius') # gradients are normalized by the major radius
#simulation.normalization.set('vpari','vt')
#simulation.normalization.set('vpare','vt')
#simulation.normalization.set('mui','mu0') # ion magnetic moment normalized by
#simulation.normalization.set('mue','mu0') # electron magnetic moment normalized by

sim_frames = simulation.available_frames['ion'] # you can check the available frames for each data type like ion_M0, ion_BiMaxwellian, etc.)
print("%g time frames available (%g to %g)"%(len(sim_frames),sim_frames[0],sim_frames[-1]))

In [ ]:
pygkyl.plot_utils.plot_2D_cut(simulation=simulation, 
                              fieldnames=['Bmag','phi','ne','ni','Te','Ti'],
                              cut_dir='xy',
                              cut_coord=0,
                              time_frame=sim_frames[-1],
                              clim=[[1.8,2.3],[0,300],[1e17,3e18],[1e17,5e18],[0,100],[0,300]],
                              xlim =[0.04,0.12]
)

In [ ]:
outfilename = 'gyraze_input_data.h5'
gyraze = pygkyl.GyrazeInterface(simulation, filter_negativity=False, number_datasets=True, outfilename=outfilename)
tf=sim_frames[-1]
xmin=0.045
xmax=0.115
Nxsample=24
Nysample=24
alphadeg=0.3
zplane='both'
verbose=False
gyraze.generate(tf, xmin, xmax, Nxsample, Nysample, alphadeg, zplane, verbose)

In [ ]:
gyraze.dataset.attributes['B'].v0

In [ ]:
_ = gyraze.verify_h5_data(verbose=False,Nsamp=24)

In [ ]:
sidx = 0
gyraze.extract_dataset_as_files(group_name=f'{sidx:06d}',output_dir='gyraze_data')

In [ ]:
attributes = ['B0','phi0','ni0','ne0','Ti0','Te0','gamma0','nioverne','TioverTe']
values = []
for attr in attributes:
    values.append(gyraze.collect_attribute(attr))

fig, axs = plt.subplots(3,3, figsize=(10,10))
axs = axs.flatten()
for i, attr in enumerate(attributes):
    axs[i].hist(values[i], bins=32)
    axs[i].set_title(attr)
plt.tight_layout()